In [20]:
import os
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from sklearn import metrics
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
%matplotlib inline

In [21]:
os.getcwd()

'C:\\Users\\thetr'

In [22]:
# load the data set
df = pd.read_csv('Py-files/wifi_locations_master.csv', encoding = "ISO-8859-1")
df.head()

,OBJECTID,Borough,Type,Provider,Name,Location,Latitude,Longitude,X,Y,...,Neighborhood Tabulation Area (NTA),Council Distrcit,Zipcode,BoroCD,Census Tract,BCTCB2010,BIN,BBL,DOITT_ID,"Location (Lat, Long)"
0,8641,4,Free,Spot On Networks,QUEENS BRIDGE - JACOB A. RIIS Settlement House,10-25 41 AVENUE,40.755727,-73.944583,999603.2262,214613.2746,...,Queensbridge-Ravenswood-Long Island City,26,11101,401,25,25,4433386,4004700100,4746,"(40.755727, -73.944583)"
1,8642,4,Free,Spot On Networks,QUEENS BRIDGE - JACOB A. RIIS Settlement House,10-43 41 AVENUE,40.755333,-73.944131,999728.5438,214469.8070,...,Queensbridge-Ravenswood-Long Island City,26,11101,401,25,25,4433386,4004700100,4747,"(40.755333, -73.944131)"
2,8643,4,Free,Spot On Networks,QUEENS BRIDGE - JACOB A. RIIS Settlement House,10-05 41 AVENUE,40.755751,-73.945166,999441.7012,214621.9169,...,Queensbridge-Ravenswood-Long Island City,26,11101,401,25,25,4433386,4004700100,4748,"(40.755751, -73.945166)"
3,8644,4,Free,Spot On Networks,QUEENS BRIDGE - JACOB A. RIIS Settlement House,10-05 41 AVENUE,40.755751,-73.945166,999441.7012,214621.9169,...,Queensbridge-Ravenswood-Long Island City,26,11101,401,25,25,4433386,4004700100,4749,"(40.755751, -73.945166)"
4,8645,4,Free,Spot On Networks,QUEENS BRIDGE - JACOB A. RIIS Settlement House,10-05 41 AVENUE,40.755751,-73.945166,999441.7012,214621.9169,...,Queensbridge-Ravenswood-Long Island City,26,11101,401,25,25,4433386,4004700100,4750,"(40.755751, -73.945166)"


In [23]:
# represent points consistently as (lat, lon)
coords = df.as_matrix(columns=['Latitude', 'Longitude'])

# define the calculation factor in one radian
factor_per_radian = 6371

# define epsilon as 0.435 * coefficient of calculation * converted to radians for use by haversine
epsilon = 1.5/factor_per_radian

C:\Users\thetr\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [24]:
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))

Clustered 3,345 points down to 41 clusters, for 98.8% compression in 0.12 seconds
Silhouette coefficient: 0.074


In [25]:
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])

In [26]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

centermost_points = clusters.map(get_centermost_point)

In [27]:
lats, lons = zip(*centermost_points)
rep_points = pd.DataFrame({'lon':lons, 'lat':lats})
rep_points.head()

,lon,lat
0,-73.949855,40.748497
1,-74.100012,40.579839
2,-73.839181,40.874777
3,-74.114822,40.634509
4,-73.844300,40.743200


In [28]:
rep_points.to_csv('Py-files/wifi_node_locations.csv', encoding = "ISO-8859-1")
rs.tail()

,OBJECTID,Borough,Type,Provider,Name,Location,Latitude,Longitude,X,Y,...,Neighborhood Tabulation Area (NTA),Council Distrcit,Zipcode,BoroCD,Census Tract,BCTCB2010,BIN,BBL,DOITT_ID,"Location (Lat, Long)"
36,397,4,Free,QPL,South Ozone Park,128-16 ROCKAWAY BOULEVARD,40.674777,-73.809262,1.037158e+06,185173.4397,...,South Ozone Park,28,11420,410,818,818,4254814,4117500057,484,"(40.6747769261, -73.8092618174)"
37,420,5,Free,NYPL,Great Kills,56 GIFFORDS LANE,40.552423,-74.151112,9.422569e+05,140575.4151,...,Great Kills,51,10308,503,14604,14604,5070285,5054360019,509,"(40.5524232628, -74.1511116431)"
38,826,5,Limited Free,SPECTRUM,Greenbelt Recreation Center,Parking area and ballfield,40.591904,-74.138573,9.457641e+05,154953.4424,...,New Springville-Bloomfield-Travis,50,10314,502,27302,27302,0,5019550001,832,"(40.591904, -74.138573)"
39,1755,4,Limited Free,SPECTRUM,Alley Pond Park,Tennis Courts and Ball Field,40.739547,-73.735920,1.057431e+06,208823.8344,...,Bellerose,23,11426,413,1567,1567,0,4078600020,19,"(40.739547, -73.73592)"
40,1336,5,Limited Free,SPECTRUM,Wolfe's Pond Park,Back of comfort station and picnic area,40.518167,-74.189765,9.314883e+05,128115.9810,...,Annadale-Huguenot-Prince's Bay-Eltingville,51,10309,503,198,198,0,5065960001,1195,"(40.518167, -74.189765)"
